In [10]:
import branca.colormap as cm  # Add this import

# Rest of your imports
import geopandas as gpd
from shapely.geometry import shape
import openrouteservice
from openrouteservice import exceptions
import pandas as pd
import folium
from folium.plugins import HeatMap
import os

# Initialize ORS client and define generate_isochrones function
# (As shown in the previous section)

In [6]:
flats_with_pop=gpd.read_file('/home/silas/projects/msc_thesis/data/derived_data/flats_population.gpkg')
flats_with_pop.to_crs(epsg=4326, inplace=True)
rcps=gpd.read_file('/home/silas/projects/msc_thesis/data/raw_data/geodata_stadt_Zuerich/recycling_sammelstellen/data/stzh.poi_sammelstelle_view.shp')
rcps.to_crs(epsg=4326, inplace=True)

In [11]:
def generate_isochrones(client, rcps, time_range):
    """
    Generate isochrones for each RCP point.

    Parameters:
    - client: ORS client instance.
    - rcps: GeoDataFrame with RCP points.
    - time_range: List of time ranges in seconds.

    Returns:
    - GeoDataFrame with isochrones and time attribute.
    """
    isochrones_list = []

    for idx, row in rcps.iterrows():
        lon, lat = row.geometry.x, row.geometry.y
        for time in time_range:
            try:
                params = {
                    "locations": [[lon, lat]],
                    "range": [time],
                    "range_type": "time",
                    "units": "m",
                    "location_type": "start",
                    "smoothing": 0.3,
                    "profile": "foot-walking",
                }
                isochrone = client.isochrones(**params)
                for feature in isochrone['features']:
                    isochrones_list.append({
                        'geometry': shape(feature['geometry']),
                        'rcp_id': row['standort_i'],
                        'time': time / 60})
                    print(f"Generated isochrone for RCP ID {row['standort_i']} for {time} minutes.")

            except exceptions.ApiError as e:
                print(f"API error for RCP ID {row['standort_i']} at time {time} seconds: {e}")
            except Exception as e:
                print(f"Unexpected error for RCP ID {row['standort_i']} at time {time} seconds: {e}")

    isochrones_gdf = gpd.GeoDataFrame(isochrones_list, crs="EPSG:4326")
    return isochrones_gdf

def get_ors_client():
    """Initialize OpenRouteService client"""
    api_key = os.getenv("ORS_API_KEY")
    if not api_key:
        logger.error("OpenRouteService API key not found.")
        sys.exit(1)
    return openrouteservice.Client(key=api_key)

client = get_ors_client()

# Example usage
time_range = list(range(60, 660, 60))  # [60, 120, ..., 600] seconds
isochrones_gdf = generate_isochrones(client, rcps, time_range)
print(isochrones_gdf)

Generated isochrone for RCP ID 42964.0 for 60 minutes.
Generated isochrone for RCP ID 42964.0 for 120 minutes.
Generated isochrone for RCP ID 42964.0 for 180 minutes.
Generated isochrone for RCP ID 42964.0 for 240 minutes.
Generated isochrone for RCP ID 42964.0 for 300 minutes.
Generated isochrone for RCP ID 42964.0 for 360 minutes.
Generated isochrone for RCP ID 42964.0 for 420 minutes.
Generated isochrone for RCP ID 42964.0 for 480 minutes.
Generated isochrone for RCP ID 42964.0 for 540 minutes.
Generated isochrone for RCP ID 42964.0 for 600 minutes.
Generated isochrone for RCP ID 42970.0 for 60 minutes.
Generated isochrone for RCP ID 42970.0 for 120 minutes.
Generated isochrone for RCP ID 42970.0 for 180 minutes.
Generated isochrone for RCP ID 42970.0 for 240 minutes.
Generated isochrone for RCP ID 42970.0 for 300 minutes.
Generated isochrone for RCP ID 42970.0 for 360 minutes.
Generated isochrone for RCP ID 42970.0 for 420 minutes.
Generated isochrone for RCP ID 42970.0 for 480 min

/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/silas/miniconda3/envs/geo_env/lib/python3.13/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


KeyboardInterrupt: 

In [ ]:
isochrones_gdf.to_file('/home/silas/projects/msc_thesis/data/derived_data/isochrones_1-10min.gpkg', driver='GPKG')
print(isochrones_gdf.crs)


EPSG:4326


In [ ]:
isochrones_gdf

,geometry,rcp_id,time
0,"POLYGON ((8.55975 47.37164, 8.55976 47.37108, ...",42964.0,1.0
1,"POLYGON ((8.55861 47.37082, 8.55865 47.3707, 8...",42964.0,2.0
2,"POLYGON ((8.55771 47.37156, 8.55746 47.37052, ...",42964.0,3.0
3,"POLYGON ((8.5567 47.37022, 8.55684 47.36989, 8...",42964.0,4.0
4,"POLYGON ((8.5557 47.37144, 8.55577 47.3698, 8....",42964.0,5.0
...,...,...,...
1755,"POLYGON ((8.48404 47.38056, 8.4843 47.37957, 8...",60840.0,6.0
1756,"POLYGON ((8.48347 47.37995, 8.48463 47.37732, ...",60840.0,7.0
1757,"POLYGON ((8.48146 47.38413, 8.48234 47.38147, ...",60840.0,8.0
1758,"POLYGON ((8.48191 47.37975, 8.48343 47.37682, ...",60840.0,9.0


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a folium map centered around Zurich
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13)

# Add isochrones to the map
for _, row in isochrones_gdf.iterrows():
    folium.GeoJson(row['geometry'], name=f"Isochrone {row['time']} min").add_to(m)

# Add recycling collection points to the map
marker_cluster = MarkerCluster().add_to(m)
for _, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['adresse'],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(marker_cluster)

# Display the map
m

In [ ]:
from shapely.ops import unary_union

def merge_isochrones_preserve_time(isochrones_gdf):
    """
    Merge isochrones preserving lower time values.

    Parameters:
    - isochrones_gdf: GeoDataFrame with isochrones and 'time' attribute.

    Returns:
    - GeoDataFrame with merged isochrones.
    """
    # Ensure CRS is EPSG:4326
    if isochrones_gdf.crs != "EPSG:4326":
        isochrones_gdf = isochrones_gdf.to_crs(epsg=4326)

    # Sort isochrones by 'time' ascending
    isochrones_sorted = isochrones_gdf.sort_values(by='time')

    merged_isochrones = gpd.GeoDataFrame(columns=isochrones_sorted.columns, crs="EPSG:4326")

    # Initialize an empty geometry for subtraction
    accumulated_geom = None

    for _, row in isochrones_sorted.iterrows():
        current_geom = row.geometry
        current_time = row['time']

        if accumulated_geom:
            remaining_geom = current_geom.difference(accumulated_geom)
        else:
            remaining_geom = current_geom

        if not remaining_geom.is_empty:
            new_row = row.copy()
            new_row.geometry = remaining_geom
            # Ensure the new_row GeoDataFrame has the correct CRS
            new_row = gpd.GeoDataFrame([new_row], crs="EPSG:4326")
            merged_isochrones = pd.concat([merged_isochrones, new_row], ignore_index=True)
            # Update accumulated geometry
            if accumulated_geom:
                accumulated_geom = unary_union([accumulated_geom, remaining_geom])
            else:
                accumulated_geom = remaining_geom
        print(f"Processed isochrone with time {current_time} minutes.")

    return merged_isochrones

# Merge isochrones
merged_isochrones_gdf = merge_isochrones_preserve_time(isochrones_gdf)
print(merged_isochrones_gdf)

/tmp/ipykernel_14203/1338846423.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_isochrones = pd.concat([merged_isochrones, new_row], ignore_index=True)


Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed isochrone with time 1.0 seconds.
Processed i

In [ ]:
# Function to prepare heatmap data

merged_isochrones_gdf.to_crs(epsg=4326, inplace=True)
def prepare_heatmap_data(merged_isochrones_gdf):
    """
    Prepare heatmap data from merged isochrones.

    Parameters:
    - merged_isochrones_gdf: GeoDataFrame with merged isochrones.

    Returns:
    - List of [latitude, longitude, intensity] for HeatMap.
    """
    heat_data = []
    for _, row in merged_isochrones_gdf.iterrows():
        if 'time' not in row:
            print("Missing 'time' column in merged_isochrones_gdf")
            continue
        centroid = row.geometry.centroid
        lat, lon = centroid.y, centroid.x
        intensity = row['time'] / 60  # Convert time to minutes for intensity
        heat_data.append([lat, lon, intensity])
    return heat_data


In [9]:
heat_data = prepare_heatmap_data(merged_isochrones_gdf)

# Initialize Folium map centered around Zurich
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13)

# Create a continuous color map for the heatmap
colormap = cm.linear.YlOrRd_09.scale(0, 10)
colormap.caption = 'Isochrone Time (minutes)'
colormap.add_to(m)

# Add merged isochrones to the map
for _, row in merged_isochrones_gdf.iterrows():
    folium.GeoJson(
        row['geometry'],
        name=f"Isochrone {row['time']} min",
        style_function=lambda feature, time=row['time']: {
            'fillColor': colormap(time),
            'color': colormap(time),
            'weight': 1,
            'fillOpacity': 0.9,
        }
    ).add_to(m)

# Add recycling collection points to the map using MarkerCluster
marker_cluster = MarkerCluster().add_to(m)
for _, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['adresse'],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(marker_cluster)

# Save and display the map
heatmap_path = '/home/silas/projects/msc_thesis/data/derived_data/heatmap_test_good.html'
m.save(heatmap_path)
m


NameError: name 'prepare_heatmap_data' is not defined

In [81]:
# Spatial join: assign each flat to an isochrone
joined = gpd.sjoin(flats_with_pop, merged_isochrones_gdf, how='inner', predicate='within')

# Group by isochrone time and sum the population
population_per_isochrone = joined.groupby('time')['est_pop'].sum().reset_index()

print(population_per_isochrone)

   time        est_pop
0   1.0   26082.169159
1   2.0   73673.046429
2   3.0  100040.457367
3   4.0   86979.071917
4   5.0   67480.358228
5   6.0   43289.651934
6   7.0   21737.014336
7   8.0   11153.744589
8   9.0    6918.020630
9  10.0    3802.011875


In [87]:
import geopandas as gpd

# Step 1: Verify 'time' column exists in merged_isochrones_gdf
if 'time' not in merged_isochrones_gdf.columns:
    raise KeyError("'time' column is missing in merged_isochrones_gdf")

# Step 2: Spatial join with a left join to retain all flats
joined = gpd.sjoin(flats_with_pop, merged_isochrones_gdf[['geometry', 'time']], how='left', predicate='within')

# Step 3: Assign a default high time value to unserved flats
iso_threshold = 10
joined['time'] = joined['time'].fillna(iso_threshold + 1)  # Assign a value greater than threshold

# Step 4: Filter unserved flats within the isochrone threshold
high_pop_unserved = joined[joined['time'] <= iso_threshold]

print(high_pop_unserved)

               egid   est_pop                  geometry  index_right  time
0          145044.0  2.017699  POINT (8.51729 47.36728)        333.0   2.0
1          145253.0  2.317757  POINT (8.51583 47.36683)        445.0   3.0
2          144667.0  2.386364  POINT (8.51466 47.36713)        371.0   3.0
3          144668.0  2.386364  POINT (8.51458 47.36708)        371.0   3.0
4          144669.0  2.567010  POINT (8.51428 47.36681)        334.0   2.0
...             ...       ...                       ...          ...   ...
240526  302061950.0  1.885714  POINT (8.51024 47.39141)        703.0   5.0
240527  302061950.0  2.514286  POINT (8.51024 47.39141)        703.0   5.0
240528  302061950.0  2.514286  POINT (8.51024 47.39141)        703.0   5.0
240529  302061950.0  1.885714  POINT (8.51024 47.39141)        703.0   5.0
240530  302061950.0  1.885714  POINT (8.51024 47.39141)        703.0   5.0

[238908 rows x 5 columns]


In [89]:
import geopandas as gpd
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Define isochrone threshold (e.g., 10 minutes)
iso_threshold = 10

# Filter population data outside current collection points' isochrones
served_flats = gpd.sjoin(flats_with_pop, rcps, how='inner', predicate='within')
unserved = flats_with_pop[~flats_with_pop.index.isin(served_flats.index)]

# Further filter unserved flats within the isochrone threshold
# Assuming 'time' column represents isochrone time in minutes

# Prepare data for clustering
coords = high_pop_unserved.geometry.centroid
X = pd.DataFrame({'x': coords.x, 'y': coords.y, 'population': high_pop_unserved['est_pop']})

# Determine number of clusters (e.g., 5 new points)
k = 1
kmeans = KMeans(n_clusters=k, random_state=0)
kmeans.fit(X[['x', 'y']])
X['cluster'] = kmeans.labels_

# Calculate cluster centers weighted by population
cluster_centers = X.groupby('cluster').apply(
    lambda df: pd.Series({
        'x': (df['x'] * df['population']).sum() / df['population'].sum(),
        'y': (df['y'] * df['population']).sum() / df['population'].sum()
    })
).reset_index()

# Create GeoDataFrame for new collection points
new_points = gpd.GeoDataFrame(
    cluster_centers,
    geometry=gpd.points_from_xy(cluster_centers['x'], cluster_centers['y']),
    crs="EPSG:4326"
)

print(new_points)

# Optional: Plotting
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13)

# Add existing collection points
for _, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['adresse'],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(m)

# Add new collection points
for _, row in new_points.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup='New Collection Point',
        icon=folium.Icon(color='blue', icon='plus', prefix='fa')
    ).add_to(m)

m.save('/home/silas/projects/msc_thesis/data/derived_data/new_collection_points.html')
m

/tmp/ipykernel_14203/2697608912.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords = high_pop_unserved.geometry.centroid
/tmp/ipykernel_14203/2697608912.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cluster_centers = X.groupby('cluster').apply(


   cluster         x          y                 geometry
0        0  8.528899  47.385287  POINT (8.5289 47.38529)


In [ ]:
import geopandas as gpd
import pandas as pd
from sklearn.cluster import DBSCAN
import folium

# Step 2: Prepare data for clustering
coords = high_pop_unserved.geometry.centroid
X = pd.DataFrame({
    'x': coords.x,
    'y': coords.y,
    'population': high_pop_unserved['est_pop']
})

# Step 3: Apply DBSCAN clustering
db = DBSCAN(eps=0.01, min_samples=5).fit(X[['x', 'y']])
X['cluster'] = db.labels_

# Remove noise points
clusters = X[X['cluster'] != -1]

# Step 4: Calculate cluster centers weighted by population
cluster_centers = clusters.groupby('cluster').apply(
    lambda df: pd.Series({
        'x': (df['x'] * df['population']).sum() / df['population'].sum(),
        'y': (df['y'] * df['population']).sum() / df['population'].sum()
    })
).reset_index()

# Step 5: Create GeoDataFrame for new collection points
new_points = gpd.GeoDataFrame(
    cluster_centers,
    geometry=gpd.points_from_xy(cluster_centers['x'], cluster_centers['y']),
    crs="EPSG:4326"
)

print(new_points)

# Step 6: Plotting
m = folium.Map(location=[47.3769, 8.5417], zoom_start=13)

# Add existing collection points
for _, row in rcps.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['adresse'],
        icon=folium.Icon(color='green', icon='recycle', prefix='fa')
    ).add_to(m)

# Add new collection points
for _, row in new_points.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup='New Collection Point',
        icon=folium.Icon(color='blue', icon='plus', prefix='fa')
    ).add_to(m)

m.save('/home/silas/projects/msc_thesis/data/derived_data/new_collection_points_dbscan.html')
m

/tmp/ipykernel_14203/2243314739.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords = high_pop_unserved.geometry.centroid


: 

In [83]:
unserved

,egid,est_pop,geometry
0,145044.0,2.017699,POINT (8.51729 47.36728)
1,145253.0,2.317757,POINT (8.51583 47.36683)
2,144667.0,2.386364,POINT (8.51466 47.36713)
3,144668.0,2.386364,POINT (8.51458 47.36708)
4,144669.0,2.567010,POINT (8.51428 47.36681)
...,...,...,...
240526,302061950.0,1.885714,POINT (8.51024 47.39141)
240527,302061950.0,2.514286,POINT (8.51024 47.39141)
240528,302061950.0,2.514286,POINT (8.51024 47.39141)
240529,302061950.0,1.885714,POINT (8.51024 47.39141)
